# Import section

In [2]:
! git clone https://github.com/Icon1cc/NLP-Polimi-Project.git

fatal: destination path 'NLP-Polimi-Project' already exists and is not an empty directory.


In [3]:
pip install datasets

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.util import ngrams
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords
from datasets import load_dataset
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import Dataset, DatasetDict
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import BertTokenizer, BertLMHeadModel
from tokenizers import BertWordPieceTokenizer
from transformers import pipeline, BertTokenizer, BertForQuestionAnswering, BertTokenizerFast
from transformers import TrainingArguments, Trainer

In [6]:
# Load the dataset
dataset = load_dataset('arrow', data_files='./NLP-Polimi-Project/Practice Models/Rishabh/dataset/train/data-00000-of-00001.arrow')
df = dataset['train'].to_pandas()
df = df.iloc[:, :-1]

In [7]:
# Dataset length
len(df)

33955

In [8]:
df.head(5)

,input,output
0,What is the relationship between very low Mg2+...,Very low Mg2+ levels correspond to low PTH lev...
1,What leads to genitourinary syndrome of menopa...,Low estradiol production leads to genitourinar...
2,What does low REM sleep latency and experienci...,Low REM sleep latency and experiencing halluci...
3,What are some possible causes of low PTH and h...,"PTH-independent hypercalcemia, which can be ca..."
4,How does the level of anti-müllerian hormone r...,The level of anti-müllerian hormone is directl...


# Import BERT Model

In [9]:
model_name = 'bert-base-uncased'

In [10]:
# Load the model
model = BertForQuestionAnswering.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predic

In [11]:
# Load the tokenizer
tokenizer = BertTokenizerFast.from_pretrained(model_name)

# Preprocess data

In [12]:
# Split dataset into training, validation, and test sets
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [ ]:
# Define preprocessing function
def preprocess_data(df):
    inputs = []
    labels = []
    for _, row in df.iterrows():
        question_text = row['input']
        answer_text = row['output']

        # Tokenize question and answer
        inputs.append(tokenizer.encode(question_text, answer_text, padding='max_length', truncation=True, max_length=512, return_tensors='pt').squeeze())
        # Assuming start_positions are used as labels for answer spans
        labels.append(tokenizer.encode(answer_text, padding='max_length', truncation=True, max_length=512, return_tensors='pt').squeeze())

    return inputs, labels

In [ ]:
# Preprocess training, validation, and test data
train_inputs, train_labels = preprocess_data(train_df)
val_inputs, val_labels = preprocess_data(val_df)
test_inputs, test_labels = preprocess_data(test_df)

In [ ]:
# Create Dataset objects
train_dataset = Dataset.from_dict({'input_ids': train_inputs, 'start_positions': train_labels})
val_dataset = Dataset.from_dict({'input_ids': val_inputs, 'start_positions': val_labels})
test_dataset = Dataset.from_dict({'input_ids': test_inputs, 'start_positions': test_labels})

In [ ]:
# Create a DatasetDict to store train, validation, and test datasets
data = DatasetDict()
data['train'] = train_dataset
data['validation'] = val_dataset
data['test'] = test_dataset


In [ ]:
# Define tokenization function
def tokenize_function(examples):
    return tokenizer(examples)


In [ ]:
print(train_dataset[0])

{'input_ids': [101, 2054, 2003, 1996, 3114, 2005, 1996, 5915, 1011, 14447, 1997, 2895, 1997, 1996, 4607, 11439, 20303, 2015, 1997, 2358, 9331, 29598, 24163, 27631, 8740, 23446, 1029, 102, 2054, 2003, 1996, 3114, 2005, 1996, 5915, 1011, 14447, 1997, 2895, 1997, 1996, 4607, 11439, 20303, 2015, 1997, 2358, 9331, 29598, 24163, 27631, 8740, 23446, 1029, 1996, 4607, 11439, 20303, 2015, 2024, 3653, 29021, 1010, 2029, 2003, 2339, 2027, 2031, 1037, 5915, 1011, 14447, 1997, 2895, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
tokenized_data = data.map(lambda examples: tokenizer(examples['input_ids']), batched=True)

Map:   0%|          | 0/27164 [00:00<?, ? examples/s]

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

# Preprocess 2


In [24]:
# Split dataset into training, validation, and test sets
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [13]:
# The input_ids provide the tokenized input to the model
# The start_position indicates where in that input the answer should start
def preprocess_data(df):
    inputs = []
    start_positions = []
    # Iterating through dataframe rows
    for _, row in df.iterrows():
        # Extracting input and output text
        question_text = row['input']
        answer_text = row['output']

        # Tokenize question and answer
        encoding = tokenizer(question_text, answer_text, padding='max_length', truncation=True, max_length=512, return_tensors='pt')

        # Append to lists
        inputs.append(encoding['input_ids'])

        # Find the start position of the answer in the tokenized input
        start_idx = encoding.char_to_token(0, len(question_text))
        start_positions.append(start_idx)

    return inputs, start_positions


In [14]:
# Preprocess training, validation, and test data
train_inputs, train_start_positions = preprocess_data(train_df)
val_inputs, val_start_positions = preprocess_data(val_df)
test_inputs, test_start_positions = preprocess_data(test_df)


In [15]:
# Create Dataset objects
train_dataset = Dataset.from_dict({'input_ids': train_inputs, 'start_positions': train_start_positions})
val_dataset = Dataset.from_dict({'input_ids': val_inputs, 'start_positions': val_start_positions})
test_dataset = Dataset.from_dict({'input_ids': test_inputs, 'start_positions': test_start_positions})

In [16]:
# Create a DatasetDict to store train, validation, and test datasets
data = DatasetDict()
data['train'] = train_dataset
data['validation'] = val_dataset
data['test'] = test_dataset

In [19]:
# Tokenize data
tokenized_data = data.map(lambda examples: tokenizer(examples['input_ids']), batched=True)

Map:   0%|          | 0/27164 [00:00<?, ? examples/s]

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [1]:
!pip install transformers==4.30

In [18]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 2.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [21]:
pip install transformers[torch]

In [23]:
training_args = TrainingArguments(
    "cool_trainer_name",
    per_device_train_batch_size=16
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data['train'],
    eval_dataset=data['validation'],
)


In [ ]:

# Train the model
trainer.train()